In [ ]:
# 実行後、ランタイムを再起動してください
!pip install dwave-ocean-sdk

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 798kB 9.0MB/s 
     |████████████████████████████████| 399kB 38.9MB/s 
     |████████████████████████████████| 204kB 38.1MB/s 
     |████████████████████████████████| 788kB 22.5MB/s 
     |████████████████████████████████| 7.0MB 18.1MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 409kB 36.0MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 3.3MB 22.4MB/s 
     |████████████████████████████████| 14.2MB 208kB/s 
     |████████████████████████████████| 1.0MB 28.0MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
import numpy as np

# 都市数
N = 5

# 都市間の移動距離
Q = np.array([[1000, 20, 20, 50, 40],
              [30, 1000, 10, 30, 20],
              [20, 10, 1000, 30, 20],
              [50, 30, 20, 1000, 10],
              [40, 20, 20, 10, 1000]])

In [ ]:
from pyqubo import Array, Constraint, Placeholder
x = Array.create('x', shape=(N, N), vartype='BINARY')

# 目的関数
cost = 0
for i in range(N):
  for j in range(N):
    cost = cost + Q[i][j]*x[i][j]

# 制約条件1：各都市に1回は訪問しなければならない
constr_1 = 0
for i in range(N):
  constr_1 += (np.sum(x[i])-1)**2

# 制約条件2：1度に訪れる都市は1つでなければならない
constr_2 = 0
for i in range(N):
  constr_2 += (np.sum(x.T[i])-1)**2

# コスト関数
cost_func = cost + Placeholder('lam')*Constraint(constr_1, label='constr_1') + Placeholder('lam')*Constraint(constr_2, label='constr_2')
model = cost_func.compile()

In [ ]:
feed_dict = {'lam': 100.0} # penalty係数
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [ ]:
# dict形式のQUBO
qubo

{('x[0][0]', 'x[0][0]'): 800.0,
 ('x[0][0]', 'x[0][1]'): 200.0,
 ('x[0][0]', 'x[1][0]'): 200.0,
 ('x[0][1]', 'x[0][1]'): -180.0,
 ('x[0][1]', 'x[1][1]'): 200.0,
 ('x[0][2]', 'x[0][0]'): 200.0,
 ('x[0][2]', 'x[0][1]'): 200.0,
 ('x[0][2]', 'x[0][2]'): -180.0,
 ('x[0][2]', 'x[1][2]'): 200.0,
 ('x[0][3]', 'x[0][0]'): 200.0,
 ('x[0][3]', 'x[0][1]'): 200.0,
 ('x[0][3]', 'x[0][2]'): 200.0,
 ('x[0][3]', 'x[0][3]'): -150.0,
 ('x[0][3]', 'x[1][3]'): 200.0,
 ('x[0][4]', 'x[0][0]'): 200.0,
 ('x[0][4]', 'x[0][1]'): 200.0,
 ('x[0][4]', 'x[0][2]'): 200.0,
 ('x[0][4]', 'x[0][3]'): 200.0,
 ('x[0][4]', 'x[0][4]'): -160.0,
 ('x[0][4]', 'x[1][4]'): 200.0,
 ('x[1][0]', 'x[1][0]'): -170.0,
 ('x[1][0]', 'x[1][1]'): 200.0,
 ('x[1][1]', 'x[1][1]'): 800.0,
 ('x[1][2]', 'x[1][0]'): 200.0,
 ('x[1][2]', 'x[1][1]'): 200.0,
 ('x[1][2]', 'x[1][2]'): -190.0,
 ('x[1][3]', 'x[1][0]'): 200.0,
 ('x[1][3]', 'x[1][1]'): 200.0,
 ('x[1][3]', 'x[1][2]'): 200.0,
 ('x[1][3]', 'x[1][3]'): -170.0,
 ('x[1][4]', 'x[1][0]'): 200.0,
 

# D-Waveマシンでの実行

In [ ]:
token = "D-Wave Leapのtokenを入力してください"　
endpoint = 'https://cloud.dwavesys.com/sapi/'

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
dw_sampler = DWaveSampler(solver='DW_2000Q_6', token=token, endpoint=endpoint)

In [ ]:
sampler = EmbeddingComposite(dw_sampler)

In [ ]:
sampleset = sampler.sample_qubo(qubo, num_reads=10)

In [ ]:
# 結果出力
print(sampleset.record)

[([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], -930., 1, 0.)
 ([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], -900., 1, 0.)
 ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], -880., 1, 0.)
 ([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], -880., 1, 0.)
 ([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], -860., 1, 0.)
 ([0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], -660., 1, 0.)
 ([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], -660., 1, 0.)
 ([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], -650., 1, 0.)
 ([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], -510., 1, 0.)
 ([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], -470., 1, 0.)]


In [ ]:
# 制約条件を守っているかどうかの情報を得ることができる
decoded_samples = model.decode_sampleset(sampleset=sampleset, feed_dict=feed_dict)

# 後半の5つの出力結果が制約条件破ってる
for sample in decoded_samples:
  print(sample.constraints(only_broken=True))

{}
{}
{}
{}
{}
{'constr_1': (False, 1.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 2.0)}
{'constr_1': (False, 1.0), 'constr_2': (False, 1.0)}
{'constr_1': (False, 2.0), 'constr_2': (False, 2.0)}
{'constr_1': (False, 1.0), 'constr_2': (False, 3.0)}


In [ ]:
# コスト関数値：-930の出力結果を整形
# C→A→B→E→D→C
sampleset.record[0][0].reshape(N, N)

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]], dtype=int8)

# Openjijでの実行

In [ ]:
!pip install openjij

     |████████████████████████████████| 573kB 4.2MB/s 
     |████████████████████████████████| 1.2MB 8.1MB/s 


In [ ]:
from openjij import SQASampler
sampler = SQASampler()

In [ ]:
sampleset = sampler.sample_qubo(qubo, num_reads=10)
print(sampleset.record)

[([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], -730., 1)
 ([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0], -870., 1)
 ([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], -890., 1)
 ([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], -730., 1)
 ([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], -860., 1)
 ([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], -720., 1)
 ([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], -720., 1)
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], -560., 1)
 ([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], -680., 1)
 ([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], -720., 1)]


In [ ]:
# コスト関数値：-890の出力結果を整形
# C→A→B→E→D→C
sampleset.record[2][0].reshape(N, N)

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]], dtype=int8)